In [1]:
# day 16
import re
import numpy as np
from math import factorial

f = open("data/data16.txt", "r")
lines = f.readlines()
lines = [l[:-1] for l in lines]
line = lines[0]

class Node:
    def __init__(self, ind, name, flow):
        self.ind = ind
        self.name = name
        self.flow = flow
        self.next = []
        self.open = False
        self.visited = False

    def open_valve(self):
        self.open = True

    def visit_valve(self):
        self.visited = True

class Graph:
    def __init__(self, num_vertices):
        self.num_vertices = num_vertices
        self.adjacency_list = [[] for _ in range(num_vertices)]
        self.vertices = []
    
    def add_vertex(self, ind, name, flow):
        self.vertices.append(Node(ind, name, flow))
    
    def add_edge(self, u, v):
        self.adjacency_list[u].append(v)
    
    def get_adjacent_vertices(self, u):
        return self.adjacency_list[u]
    
    def get_degree(self, u):
        return len(self.adjacency_list[u])
    
    def get_num_vertices(self):
        return self.num_vertices

def init_graph(lines):
    L_valves = [re.search(r'Valve (\w+)', line).group(1) for line in lines]
    mygraph = Graph(len(lines))
    list_valves = []
    for i, line in enumerate(lines):
        valve_name = re.search(r'Valve (\w+)', line).group(1)
        flow_rate = int(re.search(r'flow rate=(\d+)', line).group(1))
        list_next = re.search(r'tunnels* leads* to valves* (.*)', line).group(1).split(", ")
        mygraph.add_vertex(i, valve_name, flow_rate)
        for newname in list_next:
            j = L_valves.index(newname)    
            mygraph.add_edge(i, j)
    return mygraph

def get_distances_from_graph(mygraph):
    N = len(mygraph.adjacency_list)
    A = np.zeros((N, N))
    for i, l in enumerate(mygraph.adjacency_list):
        for j in l:
            A[i, j] = 1

    D = np.zeros((N, N), dtype='int8')
    k = 1
    B = A
    while (D==0).any():
        D[(B>0) & (D==0)] = k
        k += 1
        B = B.dot(A)  
    for i in range(N):
        D[i, i] = 0
    return D        

def get_permutations(arr, l, r, permutations):
    if l == r:
        permutations.append(arr[:])
    else:
        for i in range(l, r):
            arr[l], arr[i] = arr[i], arr[l]
            get_permutations(arr, l+1, r, permutations)
            arr[l], arr[i] = arr[i], arr[l]  # backtrack                

In [2]:
mygraph = init_graph(lines)
D = get_distances_from_graph(mygraph)
flows = [v.flow for v in mygraph.vertices]
valves = [i for i in range(len(flows)) if flows[i]>0]

current_node = [v.name for v in mygraph.vertices].index("AA")
current_time = 30

def get_max_score(current_node, current_time, L_visited):
    #print(current_time)
    
    if current_time<0 or len(L_visited)==len(valves):
        return 0

    scores = [-10000]
    for option in valves:
        if option not in L_visited:
            next_time = current_time - (D[current_node, option]+1)
            score = get_max_score(option, next_time, L_visited + [option])
            score += next_time*flows[option]
            scores.append(score)
     
    return max(scores)

get_max_score(current_node, current_time, [])


2313

In [3]:
mygraph = init_graph(lines)
D = get_distances_from_graph(mygraph)
flows = [v.flow for v in mygraph.vertices]
valves = [i for i in range(len(flows)) if flows[i]>0]
print(valves)

def get_max_score_for2(current_1, current_2, current_t, wait1, wait2, L_visited):
    #print(current_time)
    
    if (current_t<=0) or (len(L_visited)==len(valves)):
        return 0

    scores = [-10000]
    if (wait1==0) and (wait2==0): # 1 and 2 are ready to move
        for i, opt1 in enumerate(valves):
            if opt1 not in L_visited:
                for opt2 in valves[:i]+valves[i+1:]:
                    if opt2 not in L_visited:
                        next_t1 = current_t - (D[current_1, opt1]+1)
                        next_t2 = current_t - (D[current_2, opt2]+1)
                        next_t = max(next_t1, next_t2)
                        score = get_max_score_for2(opt1, opt2, next_t, next_t - next_t1, next_t - next_t2, L_visited + [opt1, opt2])
                        score += max(0, next_t1)*flows[opt1]
                        score += max(0, next_t2)*flows[opt2]
                        scores.append(score)
                #next_t = current_t - (D[current_1, opt1]+1)
                #score = max(0, next_t)*flows[opt1]
                #scores.append(score)
        #for i, opt2 in enumerate(valves):
        #    if opt2 not in L_visited:
        #        next_t = current_t - (D[current_2, opt2]+1)
        #        score = max(0, next_t)*flows[opt2]
        #        scores.append(score)
    elif wait1==0: # wait2>0
        next_t2 = current_t - wait2 # moment when 2 is ready
        for opt1 in valves:
            if opt1 not in L_visited:
                next_t1 = current_t - (D[current_1, opt1]+1)
                next_t = max(next_t1, next_t2)
                score = get_max_score_for2(opt1, current_2, next_t, next_t - next_t1, next_t - next_t2, L_visited + [opt1])
                score += max(0, next_t1)*flows[opt1]
                scores.append(score)
    elif wait2==0: # wait1>0
        next_t1 = current_t - wait1 # moment when 1 is ready
        for opt2 in valves:
            if opt2 not in L_visited:
                next_t2 = current_t - (D[current_2, opt2]+1)
                next_t = max(next_t1, next_t2)
                score = get_max_score_for2(current_1, opt2, next_t, next_t - next_t1, next_t - next_t2, L_visited + [opt2])
                score += max(0, next_t2)*flows[opt2]
                scores.append(score)
    else:
        print("Pb: neither wait1 nor wait2 is zero")

    return max(scores)

starting_node = [v.name for v in mygraph.vertices].index("AA")
starting_time = 26
get_max_score_for2(starting_node, starting_node, starting_time, 0, 0, [])

[0, 3, 6, 8, 9, 13, 25, 35, 36, 37, 46, 48, 49, 50, 56]


KeyboardInterrupt: 